# Hunting - Azure Resources Provisioning

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.8<br>
__Apache Spark Version:__ 3.1<br>
__Required Packages:__ No<br>
__Platforms Supported:__  Azure Synapse Analytics
     
__Data Source Required:__ No 
    
### Description
This notebook provides step-by-step instructions and sample code to provision Azure Key Vault.<br>
*** Please run the cells sequentially to avoid errors.  Please do not use "run all cells". *** <br>

## Table of Contents
1. Warm-up
2. Azure Authentication
3. Create Azure Key Vault

## 1. Warm-up

In [ ]:
# Load Python libraries that will be used in this notebook
from azure.identity import AzureCliCredential, DefaultAzureCredential, ClientSecretCredential, DeviceCodeCredential
from azure.core.exceptions import  HttpResponseError 

from datetime import datetime, timezone, timedelta
import pandas as pd
import json
import ipywidgets
from IPython.display import display, HTML, Markdown

## 2. Azure Authentication

In [ ]:
credential = DeviceCodeCredential()

## 3. Create Azure Key Vault

In [ ]:
akv_json_string = """
{
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
    "contentVersion": "1.0.0.0",
    "metadata": {
        "_generator": {
            "name": "bicep",
            "version": "0.4.1008.15138",
            "templateHash": "9483075037702199064"
        }
    },
    "parameters": {
        "keyVaultName": {
            "type": "string",
            "defaultValue": "kvforsentinelhunting",
            "metadata": {
                "description": "Specifies the name of the key vault."
            }
        },
        "location": {
            "type": "string",
            "defaultValue": "[resourceGroup().location]",
            "metadata": {
                "description": "Specifies the Azure location where the key vault should be created."
            }
        },
        "enabledForDeployment": {
            "type": "bool",
            "defaultValue": false,
            "metadata": {
                "description": "Specifies whether Azure Virtual Machines are permitted to retrieve certificates stored as secrets from the key vault."
            }
        },
        "enableSoftDelete": {
            "type": "bool",
            "defaultValue": false,
            "metadata": {
                "description": "Specifies whether the soft delete functionality is enabled for this key vault."
            }
        },
        "enabledForDiskEncryption": {
            "type": "bool",
            "defaultValue": false,
            "metadata": {
                "description": "Specifies whether Azure Disk Encryption is permitted to retrieve secrets from the vault and unwrap keys."
            }
        },
        "enabledForTemplateDeployment": {
            "type": "bool",
            "defaultValue": false,
            "metadata": {
                "description": "Specifies whether Azure Resource Manager is permitted to retrieve secrets from the key vault."
            }
        },
        "tenantId": {
            "type": "string",
            "defaultValue": "[subscription().tenantId]",
            "metadata": {
                "description": "Specifies the Azure Active Directory tenant ID that should be used for authenticating requests to the key vault. Get it by using Get-AzSubscription cmdlet."
            }
        },
        "spObjectId": {
            "type": "string",
            "metadata": {
                "description": "Specifies the object ID of Synapse workspace."
            }
        },
        "userObjectId": {
            "type": "string",
            "metadata": {
                "description": "Specifies the object ID of a user in the Azure Active Directory tenant for the vault. The object ID must be unique for the list of access policies. Get it by using Get-AzADUser or Get-AzADServicePrincipal cmdlets."
            }
        },
        "keysPermissions": {
            "type": "array",
            "defaultValue": ["list"],
            "metadata": {
                "description": "Specifies the permissions to keys in the vault. Valid values are: all, encrypt, decrypt, wrapKey, unwrapKey, sign, verify, get, list, create, update, import, delete, backup, restore, recover, and purge."
            }
        },
        "secretsPermissions": {
            "type": "array",
            "defaultValue": ["list", "get", "set"],
            "metadata": {
                "description": "Specifies the permissions to secrets in the vault. Valid values are: all, get, list, set, delete, backup, restore, recover, and purge."
            }
        },
        "skuName": {
            "type": "string",
            "defaultValue": "standard",
            "allowedValues": ["standard", "premium"],
            "metadata": {
                "description": "Specifies whether the key vault is a standard vault or a premium vault."
            }
        },
        "clientIdName": {
            "type": "string",
            "defaultValue": "clientid",
            "metadata": {
                "description": "Specifies the name of the secret that you want to create."
            }
        },
        "clientIdValue": {
            "type": "secureString",
            "metadata": {
                "description": "Specifies the value of the secret that you want to create."
            }
        },
        "clientSecretName": {
            "type": "string",
            "defaultValue": "clientsecret",
            "metadata": {
                "description": "Specifies the name of the secret that you want to create."
            }
        },
        "clientSecretValue": {
            "type": "secureString",
            "metadata": {
                "description": "Specifies the value of the secret that you want to create."
            }
        },
        "entAppObjectId": {
            "type": "string",
            "metadata": {
                "description": "Specifies the value of Enterprise Application object id."
            }
        }
    },
    "functions": [],
    "variables": {
        "SentinelObjectId": "35700b08-27a2-4e14-b588-8dc8d05f9621"
    },
    "resources": [
        {
            "type": "Microsoft.KeyVault/vaults",
            "apiVersion": "2021-10-01",
            "name": "[parameters('keyVaultName')]",
            "location": "[parameters('location')]",
            "properties": {
                "enabledForDeployment": "[parameters('enabledForDeployment')]",
                "enabledForDiskEncryption": "[parameters('enabledForDiskEncryption')]",
                "enabledForTemplateDeployment": "[parameters('enabledForTemplateDeployment')]",
                "enableSoftDelete": "[parameters('enableSoftDelete')]",
                "tenantId": "[parameters('tenantId')]",
                "accessPolicies": [
                    {
                        "objectId": "[parameters('entAppObjectId')]",
                        "tenantId": "[parameters('tenantId')]",
                        "permissions": {
                            "secrets": "[parameters('secretsPermissions')]"
                        }
                    },
                    {
                        "objectId": "[parameters('spObjectId')]",
                        "tenantId": "[parameters('tenantId')]",
                        "permissions": {
                            "keys": "[parameters('keysPermissions')]",
                            "secrets": "[parameters('secretsPermissions')]"
                        }
                    },
                    {
                        "objectId": "[variables('SentinelObjectId')]",
                        "tenantId": "[parameters('tenantId')]",
                        "permissions": {
                            "keys": "[parameters('keysPermissions')]",
                            "secrets": "[parameters('secretsPermissions')]"
                        }
                    },
                    {
                        "objectId": "[parameters('userObjectId')]",
                        "tenantId": "[parameters('tenantId')]",
                        "permissions": {
                            "keys": "[parameters('keysPermissions')]",
                            "secrets": "[parameters('secretsPermissions')]"
                        }
                    }
                ],
                "sku": {
                    "name": "[parameters('skuName')]",
                    "family": "A"
                },
                "networkAcls": {
                    "defaultAction": "Allow",
                    "bypass": "AzureServices"
                }
            }
        },
        {
            "type": "Microsoft.KeyVault/vaults/secrets",
            "apiVersion": "2021-10-01",
            "name": "[format('{0}/{1}', parameters('keyVaultName'), parameters('clientIdName'))]",
            "properties": {
                "value": "[parameters('clientIdValue')]"
            },
            "dependsOn": ["[resourceId('Microsoft.KeyVault/vaults', parameters('keyVaultName'))]"]
        },
        {
            "type": "Microsoft.KeyVault/vaults/secrets",
            "apiVersion": "2021-10-01",
            "name": "[format('{0}/{1}', parameters('keyVaultName'), parameters('clientSecretName'))]",
            "properties": {
                "value": "[parameters('clientSecretValue')]"
            },
            "dependsOn": ["[resourceId('Microsoft.KeyVault/vaults', parameters('keyVaultName'))]"]
        }
    ]
}

"""

In [ ]:
# User inputs 
tenant_id = ""
subscription_id = ""
resource_group_name = ""
location = ""

kayvault_name = ""
synapse_workspace_object_id = ""
user_object_id = ""
ent_app_object_id = ""

In [ ]:
client_id_value = "
client_secret_value = ""

In [ ]:
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.resource.resources.models import DeploymentMode

resource_client = ResourceManagementClient(credential, subscription_id)
template_body = json.loads(akv_json_string)

rg_deployment_result = resource_client.deployments.begin_create_or_update(
    resource_group_name,
    "exampleDeployment",
    {
        "properties": {
            "template": template_body,
            "parameters": {
                "location": {
                    "value": location
                },
                "KeyvaultName": {
                    "value": kayvault_name
                },
                "spObjectId": {
                    "value": synapse_workspace_object_id
                },
                "userObjectId": {
                    "value": user_object_id
                },
                "entAppObjectId": {
                    "value": ent_app_object_id
                },
                "clientIdValue": {
                    "value": client_id_value
                },
                 "clientSecretValue": {
                    "value": client_secret_value
                }
            },
            "mode": DeploymentMode.incremental
        }
    }
)

In [ ]:
if rg_deployment_result.status() != "Succeeded":
    print(rg_deployment_result.status())
    print('Run the cell until stauts=Succeeded or when you see Failed.')
else:
    print('Done')